<a href="https://colab.research.google.com/github/lephuocdat2000/truyvandaphuongtien/blob/master/CapstoneProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#đọc file data
import pandas as pd
import numpy as np
import math
import re
import unicodedata
book_info=pd.read_json('/content/Book_info.json',encoding='utf-8')
book_info.head()

,NO,BIBID,AUTHOR,TITLE,PUBLISHER,PUBDATE,COUNTHOLDING,TOPIC,CONTENT
0,0,4386,Trần Hồng Nghi,Hướng dẫn thực hành phát triển ứng dụn...,Đại học Quốc gia Tp. Hồ Chí Min,2019,100,"[Mạng máy tính, Hệ điều hành Android]",Cung cấp những kiến thức cơ bản về công ...
1,1,4385,Lâm Đức Khải,Thiết kế luận lý số 2,Đại học Quốc gia Thành phố Hồ Chí Min,2019,100,[Kỹ thuật máy tính],None
2,2,4384,Cao Thị Nhạn,Phân tích thiết kế Hệ thống thông tin,Đại học Quốc gia Thành phố Hồ Chí Min,2019,100,[Hệ thống thông tin],None
3,3,4382,Đỗ Phúc,Phân tích Dữ liệu lớn,Đại học Quốc gia Thành phố Hồ Chí Min,2019,100,"[Hệ thống thông tin, Giáo trình]",None
4,4,4365,Nguyễn Anh Tuấn,Hệ điều hành Linux,Đại học Quốc gia TPHC,2019,100,[Hệ điều hành Linux],None


In [4]:
#Xây dựng tập từ điển 
def Dic_building(a):
  contents=book_info[a].to_numpy()
  lst_contents=[]
  dictionary=set()
  if (a!='TOPIC'):
    for content in contents:
       if content==None: words='None'
       else: 
         content=content.lower()
         words = content.replace('"', '').replace('.', '').replace("'","").replace(":","").split()
         for i,word in enumerate(words):
                 words[i] = unicodedata.normalize('NFC', word)
       lst_contents.append(words)
       dictionary.update(words)
  else:
    for content in contents:
       if content==None: words='None'
       else:
          s=''
          for i in content:
            s=s+i
          s=s.lower()
          words=s.replace('"', '').replace('.', '').replace("'","").split()
          for j,word in enumerate(words):
              words[j] = unicodedata.normalize('NFC', word)
       lst_contents.append(words)
       dictionary.update(words)
  dictionary = list(dictionary)
  return contents,lst_contents,dictionary
#Xây dựng Inverted File
def build_inverted_files(dictionary,lst_contents):
   inv_files=dict()
   for k,word in enumerate(dictionary):
      inv_files[word] = set()
      for i,content in enumerate(lst_contents):
         if word in content:
            count=content.count(word)
            tf=count/len(content)
            tup_word_tf=(i,tf)
            inv_files[word].add(tup_word_tf)
   return inv_files

#Tính toán tf-idf
def calcu_word_tfidf(inv_files,lst_contents):
    tf_idf_arr=np.zeros(len(inv_files))
    total_content=len(lst_contents)
    for i,word in enumerate(inv_files):
        k=len(inv_files[word])
        idf=math.log(total_content/(k+1))
        for tup in inv_files[word]:
            temp=list(tup)
            temp[1]*=idf
            tup=tuple(temp)
    return inv_files
#Xây dựng inverted file theo topic
def Build_invfile(a):
   contents,lst_contents,dictionary = Dic_building(a)
   inv_files = build_inverted_files(dictionary,lst_contents)
   inv_files = calcu_word_tfidf(inv_files,lst_contents)
   return contents,lst_contents,dictionary,inv_files

In [5]:
#Tính toán tfidf cho query 
def Create_tfidf_query(query,lst_contents,inv_files):
   query_words=re.findall("(\w+)", query)
   dict_query=set()
   dict_query.update(query_words)
   dict_query=list(dict_query)
   tfidf_query=np.zeros(len(dict_query))
   total_content=len(lst_contents)
   for i,word in enumerate(dict_query):
     try:
        idf=math.log(total_content/len(inv_files[word]))
     except: 
        idf=math.log(total_content)
     tfidf_query[i]=query_words.count(word)*idf/len(query_words)
   return tfidf_query,dict_query

#Thêm phần tử và kiểm tra
def Add_arr(arrs,arr):
   appear=False
   for i in arrs:
      if i[0]==arr[0]: 
        appear=True
        break
   if appear==False: arrs.append(arr)
   else: 
      for i in range(len(arrs)):
           if arrs[i][0]==arr[0]: arrs[i][1]+=[1]
   return arrs
def swap(a,b):
    temp=a
    a=b
    b=temp
    return a,b
def Sort_arr(arr):
   for i in range(0,len(arr)-1):
        for j in range(i+1,len(arr)):
           if (arr[j][1]>arr[i][1]): 
               arr[i][0],arr[j][0]=swap(arr[i][0],arr[j][0])
               arr[i][1],arr[j][1]=swap(arr[i][1],arr[j][1])
   return arr
#Mô hình tích vô hướng
def ScalarModel(tfidf_query,dict_query,inv_files,dic):
    #Tạo mảng chứa kết quả của tính vô hướng và sắp xếp
    kq_scalar=[]
    for i,word in  enumerate(dict_query):
        if word in dic:
           for tup in inv_files[word]:
              tempp=list(tup)
              tempp[1]=tempp[1]*tfidf_query[i]
              kq_scalar=Add_arr(kq_scalar,tempp)

    kq_scalar=Sort_arr(kq_scalar)
    arr_scalar=[]
    for i in range(len(kq_scalar)):
        arr_scalar.append(kq_scalar[i][0])
    return arr_scalar
#Search
def Search(query,topic):
   query=query.lower()
   contents,lst_contents,dic,inv_files=Build_invfile(topic)
   tfidf_query,dict_query=Create_tfidf_query(query,lst_contents,inv_files)
   scalar=ScalarModel(tfidf_query,dict_query,inv_files,dic)
   print(scalar)
   bibids=[]
   for i in scalar:
      bibids.append(book_info._get_value(i,'BIBID'))
   return bibids

print('Nhap tu khoa can tim:')
query=str(input())
topic='AUTHOR'
bibids=Search(query,topic)
if len(bibids)==0: print("Khong co ket qua")
else: print(bibids)




Nhap tu khoa can tim:
Le Dinh Duy
[16, 52]
[4312, 179]


In [ ]:
import numpy as np 
titile_AP={'Cơ sở dữ liệu':[1,1,1,1,1], 
'Xác suất thống kê':[1,1,0,0,0],
'Lập trình hệ thống với Java':[1,1,0,0,0],
'Các hệ cơ sở tri thức':[1,1,0,0,0],
'Lập trình C trên Windows':[1,0,0,0,0],
'Lập trình nhúng trên thiết bị di động':[1,1,1,0,0],
'Giải tích':[1,1,0,0,0],
'Đại số tuyến tính':[1,1,1,0,0],
'Mạng máy tính':[1,0,0,0,0],
'kiến trúc máy tính':[1,1,0,0,0],
'Giáo trình ngôn ngữ học máy tính':[0,0,1,0,0],
'Ngôn ngữ lập trình phần cứng Verilog':[1,0,0,0,0],
'Vũ Đức Lung':[1,1,1,1,0],
'Cao Thanh Tình':[1,0,0,0,0],
'Cao Thị Nhạn':[1,0,0,0,0],
'Phạm Thi Vương':[1,1,1,0,0],
'Nguyễn Anh Tuấn':[1,1,0,0,0],
'Nguyễn Gia Tuấn Anh':[1,0,0,0,0],
'Trần Hồng Nghi':[1,0,0,0,0],
'Phạm Thi Vương':[1,1,1,0,0],
'Đỗ Văn Nhơn':[1,1,1,1,1],
'Ngô Trung Việt':[1,0,0,0,0],
'Trí tuệ nhân tạo':[1,1,1,0,0],
'Biểu diễn tri thức':[1,1,0,0,0],
'Khoa học máy tính':[0,1,1,1,0],
'Lập trình C':[1,0,0,0,0],
'Giáo trình':[1,1,0,0,0],
'Mô hình tri thức':[1,0,0,1,0],
'Windows':[1,1,0,0,0],
'Toán học':[1,1,1,1,1],
'Thực Hành':[1,1,1,1,1],
'Lập trình':[1,1,1,1,1],
'công cụ phát triển ứng dụng trên thiết bị di động':[1,1,0,0,0],
'giáo trình của ngành công nghệ thông tin':[1,1,0,0,0],
'hệ phương trình tuyến tính và dạng toàn phương':[1,0,1,0,0],
'Giáo trình cung cấp cho sinh viên những kiến thức cơ bản về lập trình ứng dụng trên hệ điều hành Windows':[1,1,0,0,0],
'điện tử':[1,0,0,0,0],
'giới thiệu ngôn ngữ lập trình C++':[1,1,0,0,0],
'tính ma trận':[1,1,0,0,0],
'hướng dẫn giải thuật':[0,1,0,1,0]
}
arrs=titile_AP.values()
def CalCulateExpectResult(arrs):
    expect_arr=np.zeros(len(arrs))
    for i,j in enumerate(arrs):
       expect_arr[i]=np.count_nonzero(j)
    return expect_arr
expect_results=CalCulateExpectResult(arrs)


def APCalculation(arr):
    for A in np.where(np.array(arr)==1):
       AP=np.sum([(i+1)/(j+1) for i,j in enumerate(A)])
    return AP

AP_list=np.zeros(len(arrs))
for i,arr in enumerate(arrs):
    AP_list[i]=APCalculation(arr)
AP_list=AP_list/expect_results
print(AP_list)
MAP=np.mean(AP_list)
print('MAP:',MAP)    


[1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         0.33333333 1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 0.63888889 1.         1.         0.75       1.         1.
 1.         1.         1.         1.         0.83333333 1.
 1.         1.         1.         0.5       ]
MAP: 0.951388888888889
